In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langgraph.checkpoint.memory import InMemorySaver


In [2]:
load_dotenv()

PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")

model = ChatOpenAI(
    model="sonar",                     # base Sonar model
    base_url="https://api.perplexity.ai",
    api_key=PERPLEXITY_API_KEY,
    temperature=0.7
)

In [3]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [4]:
checkPoint = InMemorySaver()


In [7]:
def chat_node(state: ChatState):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}


In [8]:
graph = StateGraph(ChatState)

graph.add_node("chat_node", chat_node)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

chatbot = graph.compile()

In [10]:
chatbot.invoke({"messages": [HumanMessage(content="Hello! How are you?")]}, checkpoint=checkPoint)

{'messages': [HumanMessage(content='Hello! How are you?', additional_kwargs={}, response_metadata={}, id='f0f99edc-4bea-40b9-85de-cb02d2ed3071'),
  AIMessage(content='Hello! I\'m here to help you. How can I assist you today?  \n\nIf you are asking about the greeting "Hello! How are you?"—it\'s a common and friendly way to start a conversation in English. It can be formal or informal depending on the context, and typical responses include "I\'m good, thanks. And you?" or "I\'m fine, how about you?"[1][3][5].  \n\nThere are many other ways to say hello or greet someone in English, ranging from formal ("Good morning," "Good evening") to casual ("Hey," "Hi," "What\'s up?")[1][3][5]. For example, informal greetings among friends might include "Hey, how\'s it going?" or "Yo!"[5]. Children\'s songs often use simple and repetitive greetings like "Hello. How are you?" combined with feelings or emotions to engage learners[2][4][6].', additional_kwargs={'refusal': None}, response_metadata={'token